In [1]:
from __future__ import print_function
from keras.layers.advanced_activations import PReLU
from keras.layers import LSTM
from keras.layers import merge

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Flatten, Input, Dropout, Convolution1D, GlobalMaxPooling1D
from keras.datasets import imdb

max_features = 1000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
#print(len(X_train), 'train sequences')
#print(len(X_test), 'test sequences')

print('@@@@@@@@@@@@@@@@@@@@@@@@@')
print('y_train: \n',y_train)
print('length y_train: ', len(y_train))
print('y_test: \n',y_test)
print('length y_test: ', len(y_test))
print('@@@@@@@@@@@@@@@@@@@@@@@@@')

print('@@@@@@@@@@@@@@@@@@@@@@@@@')
print('X_train: \n',X_train)
print('length X_train: ', len(X_train))
print('X_test: \n',X_test)
print('length X_test: ', len(X_test))
print('@@@@@@@@@@@@@@@@@@@@@@@@@')
#print (X_train[0])

print(type(y_test))


Using TensorFlow backend.


Loading data...


/usr/local/lib/python2.7/dist-packages/keras/datasets/imdb.py:44: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


@@@@@@@@@@@@@@@@@@@@@@@@@
y_train: 
 [1 0 0 ..., 0 1 0]
length y_train:  25000
y_test: 
 [1 1 1 ..., 1 0 1]
length y_test:  25000
@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@
X_train: 
 [ [1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2,

In [12]:
#-----------------
print("\n X_train shape:\n",X_train.shape)
print('y_train shape: ',y_train.shape)
#----------------------------------
print('\n X_test shape:\n', X_test.shape)
print('y_test shape: ',y_test.shape)


 X_train shape:
 (25000,)
y_train shape:  (25000,)

 X_test shape:
 (25000,)
y_test shape:  (25000,)


In [13]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
#print('X_train shape:', X_train.shape)
#print('X_train', X_train)
#print('X_test shape:', X_test.shape)
#print('X_test',X_test)


print(len(y_test))
print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
print(y_train.shape)
#print(X_test)
#print(type(X_test))


print('Build model...')

Pad sequences (samples x time)
25000
(25000, 80)
(25000,)
(25000, 80)
(25000,)
Build model...


In [16]:
#-----------------
print("\n X_train shape:\n",X_train.shape, type(X_train))
print('y_train shape: ',y_train.shape)
#----------------------------------
print('\n X_test shape:\n', X_test.shape,type(X_test))
print('y_test shape: ',y_test.shape)



 X_train shape:
 (25000, 80) <type 'numpy.ndarray'>
y_train shape:  (25000,)

 X_test shape:
 (25000, 80) <type 'numpy.ndarray'>
y_test shape:  (25000,)


In [ ]:
inputs = Input(shape=(maxlen,))
x = inputs
x = Embedding(max_features, 128, dropout=0.2)(x)


#--------- LSTM layer --------------
y = inputs
y = Embedding(max_features, 128, dropout=0.2)(y)
y = LSTM(128, dropout_W=0.2, dropout_U=0.2)(y)

#----Convolution Layer------
x=Convolution1D(nb_filter=64, filter_length=3, border_mode='valid',activation='relu', subsample_length=1)(x)
x=GlobalMaxPooling1D()(x)
#---------------------------
#x = Flatten()(x)


x = merge([x,y], mode='concat')
#-------- layer of 64 relu & dropout layer------
x = Dense(64)(x)
x = PReLU()(x) # Non-linearity
#x=Dropout(0.5)(x)

#-----------------------------------------------
x = Dense(1)(x)
predictions = Activation("sigmoid")(x)




model = Model(input=inputs, output=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)